In [ ]:
import pandas as pd
import json
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [157]:
df = pd.read_json('dwts_scraper/scores-new.json')
df['performance_id'] = np.arange(df.shape[0])

df.drop( df.query(" couple== ['Dance-off','Dance Duel'] ").index, inplace = True)
df.rename(columns = {'judge': 'judge_excluded', "dance": "dance_style"}, inplace = True)
df.columns.sort_values()

Index(['broadway_show', 'cher_music', 'cirque_du_soleil_show',
       'classical_music', 'couple', 'dance_&_theme', 'dance_chosen_by',
       'dance_style', 'disney_film', 'dynamic_duo', 'elvis_music', 'era',
       'film', 'film_series', 'film_theme', 'genre', 'guest_judge',
       'horror_film_show', 'icon', 'icons', 'james_bond_film',
       'judge_excluded', 'judge_phrase', 'mentor', 'michael_buble_music',
       'motown_music', 'music', 'music_by_britney_spears',
       'music_by_janet_jackson', 'music_by_queen', 'music_from_grease',
       'musical', 'original_couple', 'performance_id', 'performance_scores',
       'professionals', 'result', 'scores', 'season', 'stevie_wonder_music',
       'team_captain', 'technical_scores', 'television_show',
       'trio_dance_partner', 'trio_partner', 'tv_show', 'villain',
       'week_title'],
      dtype='object')

In [ ]:
df['judge'] = df['judge_phrase'].str.strip(to_strip=" .\n").str.split(", ", expand = False)

In [ ]:
# week_cols = df['week_title'].str.extractall(r"Week (?P<week>\d+):?\s?(?P<week_theme>.*$)")

df['week'] = df['week_title'].str.extract(r'Week (\d+)')
df['week_theme'] = df['week_title'].str.extract(r': (.*?)$') #NaN if no Week Theme.
df['week_theme'].fillna('No Theme', inplace = True) # replace NaNs

In [ ]:
df['dance_styles'] = df['dance_style'].str.split("---")
df['primary_dance_style'] = df.dance_styles.apply(lambda x: x[0])

In [158]:
# coalesce 


# music_cols = ['elvis_music', 'classical_music', 'motown_music', 'stevie_wonder_music', 'music_by_queen', 'michael_buble_music', 'music_by_janet_jackson', 'music_from_grease', 'cher_music']

# some have the music in songs
# df['music'] = df['music'].fillna(df['songs'])


df['music'] = np.where(df['elvis_music'].notna(), df['elvis_music'] + "—Elvis Presley", df['music'])
df['music'] = np.where(df['classical_music'].notna(), df['classical_music'], df['music'])
df['music'] = np.where(df['motown_music'].notna(), df['motown_music'], df['music'])
df['music'] = np.where(df['stevie_wonder_music'].notna(), df['stevie_wonder_music'] + "—Stevie Wonder", df['music'])
df['music'] = np.where(df['music_by_britney_spears'].notna(), df['music_by_britney_spears'] + "—Brittany Spears", df['music'])
df['music'] = np.where(df['music_from_grease'].notna(), df['music_from_grease'], df['music'])
df['music'] = np.where(df['cher_music'].notna(), df['cher_music'] + "—Cher", df['music'])
df['music'] = np.where(df['music_by_queen'].notna(), df['music_by_queen'] + "—Queen", df['music'])
df['music'] = np.where(df['music_by_janet_jackson'].notna(), df['music_by_janet_jackson'] + "—Janet Jackson", df['music'])
df['music'] = np.where(df['michael_buble_music'].notna(), df['michael_buble_music'] + "—Michael Buble", df['music'])


In [119]:
# df[df['michael_buble_music'].notna()]
df[df['music'].isna()]

,couple,scores,dance_style,music,result,season,week_title,judge_phrase,elvis_music,film_theme,film,original_couple,james_bond_film,disney_film,mentor,technical_scores,performance_scores,classical_music,music_by_britney_spears,tv_show,dance_chosen_by,television_show,film_series,professionals,broadway_show,trio_dance_partner,motown_music,stevie_wonder_music,team_captain,era,guest_judge,music_from_grease,dynamic_duo,genre,horror_film_show,villain,trio_partner,icons,cher_music,judge_excluded,icon,michael_buble_music,music_by_queen,dance_&_theme,music_by_janet_jackson,cirque_du_soleil_show,musical,performance_id


In [159]:

df['songs'] = df['music'].str.split("---")
df['primary_song_phrase'] = df.songs.apply(lambda x: x[0])
df[['primary_song','primary_artist']] = df.primary_song_phrase.str.extract(r'(.*)[—―一]+(.*?)$')

In [160]:
#df.head(500)[['primary_song','primary_artist']]
df[df['primary_song'].isna()]

# drop empty scores.  Some sort of extra line in the table at https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_13)#Week_10:_Finals

,couple,scores,dance_style,music,result,season,week_title,judge_phrase,elvis_music,film_theme,film,original_couple,james_bond_film,disney_film,mentor,technical_scores,performance_scores,classical_music,music_by_britney_spears,tv_show,dance_chosen_by,television_show,film_series,professionals,broadway_show,trio_dance_partner,motown_music,stevie_wonder_music,team_captain,era,guest_judge,music_from_grease,dynamic_duo,genre,horror_film_show,villain,trio_partner,icons,cher_music,judge_excluded,icon,michael_buble_music,music_by_queen,dance_&_theme,music_by_janet_jackson,cirque_du_soleil_show,musical,performance_id,songs,primary_song_phrase,primary_song,primary_artist
2585,,,,,,13,Week 10: Finals,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.\n",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2585,[],,NaN,NaN
2776,Gabby & Alan---Heidi & Artem---Jordin & Brando...,,Freestyle---(Team Wicked),"""The Witches Are Back"" (from Hocus Pocus 2)","""The Witches Are Back"" (from Hocus Pocus 2)",31,Week 7: Halloween Night,"Carrie Ann Inaba, Len Goodman, Derek Hough, Br...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2776,"[""The Witches Are Back"" (from Hocus Pocus 2)]","""The Witches Are Back"" (from Hocus Pocus 2)",NaN,NaN


In [ ]:
# Some weeks (e.g., random have a `score` column instead of a `scores` column.  Rename that.
# this seems to have been fixed on wikipedia.

# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_20)#Weekly_scores Week 5.
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_17)#Weekly_scores Week 6.


# https://stackoverflow.com/questions/34989341/how-to-remove-nan-value-while-combining-two-column-in-panda-data-frame
# df['scores'] = df['scores'].fillna(df['score'])

In [ ]:
df.columns.sort_values()

In [ ]:
# df[df['season'] == '']

In [ ]:
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_9)#Weekly_scores Week 6 has a 'ranked order' score dance.
# Dance Offs are also a problem, some are separate tables, some are bottom rows in tables.






In [ ]:
# Some weeks have a technical score/performance score. but no scores column.

# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_11)#Weekly_scores Week 4
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_10)#Weekly_scores Week 4

# double_score = df[df['technical_score'].notna()]

# double_score[double_score.columns[~double_score.isnull().any()]]

In [ ]:
# Quite a few rows from tables that shouldn't be parsed (after end of week shows).  e.g. tributes.
# Either fix in parsing or drop here.

df.dropna(subset=['dance_style'], inplace=True)

# df[df['dance'].isna()]



In [ ]:
# season 22, week 8 has a very difficult judges score arrangement
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_22)#Week_8:_Judges'_Team-up_Challenge

#  total (9, X, 9, 9) where the final score is "america's score" and the X is the score that would have been the score from the judge
# in df['judge'] column.  gah.  but those are multi couple dances.  phew.

In [ ]:
# editing on the pages made this moot.
# Some couple columns have additional info which is in parens in the column. This creates an NA couple column.

# pattern = r'(?P<couple_alone>.*?)\((?P<additional>.*?)\)$'

# # e.g., https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_27) Week 4.  Couple (Trio Dance Partner)
# df[['couple_alone','trio_partner']] = df['couple_trio_dance_partner_'].str.extract(pattern)
# df['couple'] = df['couple'].fillna(df['couple_alone'])
# df.drop(columns='couple_alone', inplace=True)

# # https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_16) Week 5 (Professionals).
# df[['couple_alone','additional_dancers']] = df['couple_professionals_'].str.extract(pattern)
# df['couple'] = df['couple'].fillna(df['couple_alone'])
# df.drop(columns='couple_alone', inplace=True)

# # couple_team_up_judge_ from https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_22)#Week_8:_Judges'_Team-up_Challenge
# # Some of These are multi-couple dances, with a judge listed i the couple column, and then an X for the score that the judge that coached the team would have given.
# # but others are regular dances (just a different column name)
# df['couple'] = df['couple'].fillna(df['couple_team_up_judge_'])

# # couple_judge_  from https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_11)#Week_10:_Finals is much simpler (just a judge chosen dance).
# df[['couple_alone','judge_choosing_dance']] = df['couple_judge_'].str.extract(pattern)
# df['couple'] = df['couple'].fillna(df['couple_alone'])
# df.drop(columns='couple_alone', inplace=True)

# # One remains from the couple_judge_ due to a substitution which then didn't have a judge (so the regex above doesn't catch it)
# # 2341 where the couple_judge_ is Nastia & Sasha[a]
# df['couple'] = df['couple'].fillna(df['couple_judge_'])
# # df.iloc[2341]

# df.drop(columns=['couple_trio_dance_partner_', 'couple_professionals_', 'couple_judge_'], inplace=True)

# df[df['couple'].isna()]

In [ ]:
# Some have technical_score and performance_score.
# add a score_type column.
# split these out, going to be two rows, each with scores column and a score_type column.

double_scores = ( df[df['technical_scores'].notna()]
                  .drop(columns="scores")
                  .dropna(axis = 1, how = "all") )

val_cols = ['technical_scores','performance_scores']
id_cols = [ele for ele in double_scores if ele not in val_cols]
# now get two rows per
double_scores = double_scores.melt(id_vars = id_cols, value_vars = val_cols, value_name = "scores", var_name = "score_type")

single_scores = df.drop(df.index[df['technical_scores'].notna()])
# make others be score_type
single_scores['score_type'] = "single_score"
single_scores.drop(columns=['technical_scores','performance_scores'], inplace=True)

df_score_type = pd.concat([double_scores, single_scores])


In [ ]:
# df_score_type.head()

In [ ]:
df_score_type[['total_score','indiv_scores']] = df_score_type['scores'].str.extract(r'(?P<total_score>\d+)\s\((?P<indiv_scores>.*)\)')

# new_cols['judge_score_list'] = new_cols['indiv_scores'].str.split(",")


# df_new = pd.concat([df, new_cols], axis = 1)

In [ ]:
# song_cols = df['music'].str.extract(r'"(?P<song_name>.*)"—(?P<song_artist>.*)$')
# song_cols
# df["music"].str.split(" / ") Some music columns have multiples.

In [ ]:
# most are multi couple dances, with ranking scores.

df_score_type["couple_list"] = df_score_type["couple"].str.split("---")
df_score_type["couple_count"] = df_score_type["couple_list"].apply(len)



In [ ]:
# Now eliminate multi-couple dances and dances with NA total scores (mostly dance offs and others). 
#  Might still be one or two issues with total scores but not individual scores.

df_single_couple = df_score_type.loc[(df_score_type['couple_count'] == 1) & df_score_type['total_score'].notna() ].copy()

# df_single_couple.head()



In [ ]:
# df_single_couple[df_single_couple['indiv_scores'].isna()]

In [ ]:
# df_single_couple.head()

df_single_couple['judge_score_list'] = df_single_couple['indiv_scores'].str.split(", ")

# df_single_couple['judge_score_list']
df_single_couple['judge_score_count'] = df_single_couple['judge_score_list'].apply(len)

df_single_couple['judge_count'] = df_single_couple['judge'].apply(len)


# a few have a total score, but not an individual score.  e.g., s9w10.

In [ ]:
#pd.set_option('max_colwidth', None) 
#df_single_couple[df_single_couple['season'] == 31][['week','judge_phrase']]

In [ ]:
df_single_couple.loc[df_single_couple['judge_score_count'] != df_single_couple['judge_count']]

# Trouble in 
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_23)#Week_5:_Most_Memorable_Year_Night
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_31)#Week_6:_Michael_Bubl%C3%A9_Night

In [ ]:
df_final = ( df_single_couple
              #  .head(5)
               .explode(['judge','judge_score_list'])
               .rename(columns={'judge_score_list': 'judge_score'})
               .drop(columns=['indiv_scores','judge_phrase','scores'])
               .assign(judge = lambda df: df.judge.str.strip(" ."))
)

df_final.drop( df_final.query(" judge_score == 'X' ").index, inplace = True)

df_final['judge_score_id'] = np.arange(df_final.shape[0])

In [ ]:
# rename "Guest judge" with value from df_final['guest_judge']
df_final['judge'] = np.where(df_final['judge'] == 'Guest judge', df_final['guest_judge'], df_final['judge'])

# fix Julianne Hough (Night 1 only)
df_final['judge'] = np.where(df_final['judge'] == 'Julianne Hough (Night 1 only)', 'Julianne Hough', df_final['judge'])

# sorted(df_final['judge'].unique().tolist())

In [ ]:
# df_final[df_final['judge_score'] == '2']

In [ ]:
# sorted(df_final['judge_score'].unique().tolist())

In [ ]:
# df_final[df_final['judge_score'] == 'X']



In [ ]:
# df_final[df_final['judge_score'] == 'X']

In [ ]:
all_cols = df_final.columns.values.tolist()

to_remove = ['judge_excluded', 'couple_list', 'couple_count', 'judge_score_count', 'judge_count']

all_cols = [ele for ele in all_cols if ele not in to_remove]

front_cols = [ 'judge_score_id', 'judge','judge_score', 'performance_id','season','week','couple','dance_style','music','total_score' ]

remaining_cols = [ele for ele in all_cols if ele not in front_cols]

reordered = front_cols + remaining_cols

In [ ]:
df_final = df_final[reordered]

In [ ]:
df_final.to_csv("dwts_dataset.csv", index = False)

In [ ]:
sorted(df_final.columns)

# Produce tables

performances (id, song_id, dance_style_id, couple_id, season_id, week_id, week_theme_id)
dance_styles (id, name) 
songs (id, name, artist_id)
artists (id, name)
scores (id, performance_id, person_id, score)
seasons (id, season_no, year_broadcast)
week_themes (id, name, season_id)
              
couples (id, name, celebrity_person_id, pro_person_id) # maybe.

people # Note that some work needed to match up couples to people.


In [ ]:
dance_styles_series = df_final['dance_style'].unique()
dance_styles = pd.DataFrame({'id': np.arange(1, len(dance_styles_series) + 1), 'name': dance_styles_series})
dance_styles.to_csv("dwts_dataset/dance_styles.csv", index = False)

In [ ]:
couples_series = df_final['couple'].unique()
couples = pd.DataFrame({'id': np.arange(1, len(couples_series) + 1), 'name': couples_series})
couples.to_csv("dwts_dataset/couples.csv", index = False)

In [ ]:

df_final['couple_id'] = df_final.couple.factorize()[0] + 1
df_final[['couple_id','couple']].drop_duplicates().to_csv("dwts_dataset/couples.csv", index = False)

df_final['dance_style_id'] = df_final.dance_style.factorize()[0] + 1
df_final[['dance_style_id','dance_style']].drop_duplicates().to_csv("dwts_dataset/dance_styles.csv", index = False)

df_final['week_theme_id'] = df_final.week_theme.factorize()[0] + 1
df_final[['week_theme_id','week_theme']].drop_duplicates().to_csv("dwts_dataset/week_theme.csv", index = False)



In [ ]:
df_final['week_theme_id'].unique()

In [ ]:
# need to make dance_style and song many to 1.